In [1]:
import random  # 몬스터 스탯 랜덤 생성 필요

# --------------------------------------------
# Q1. Character Class 만들기
# --------------------------------------------
# Player와 Monster가 공통으로 가지는 요소(레벨, 체력, 공격력, 방어력)를
# Character라는 부모 클래스로 묶어두면 중복 코드를 줄일 수 있습니다.
class Character:
    def __init__(self, name: str, level: int, hp: int, attack: int, defense: int):
        # 이름(문자열)
        self.name = name

        # 레벨(정수)
        self.level = level

        # 체력(정수)
        self.hp = hp

        # 공격력(정수)
        self.attack = attack

        # 방어력(정수)
        self.defense = defense

    def is_alive(self) -> bool:
        # hp가 0보다 크면 살아있는 것!
        return self.hp > 0

    def take_damage(self, raw_damage: int) -> int:
        """
        상대가 준 '원래 데미지(raw_damage)'를 받아,
        내 방어력(defense)을 적용한 실제 피해량을 계산하고 hp를 깎습니다.

        실제 피해량은 최소 1로 만들겠습니다.
        (방어력이 너무 높으면 데미지가 0이 될 수도 있는데, 게임이 안 끝날 수 있어요.)
        """
        # 방어력만큼 피해를 줄임
        reduced_damage = raw_damage - self.defense

        # 실제 피해량은 최소 1로 강제
        actual_damage = max(1, reduced_damage)

        # hp 감소
        self.hp -= actual_damage

        # hp가 음수로 너무 내려가면 보기 안 좋으니 0 아래로는 그냥 둬도 되지만
        # 출력할 때 깔끔하게 보이게 하려고 0 미만이면 0으로 보정.
        
        if self.hp < 0:
            self.hp = 0

        # 실제로 깎인 피해량을 돌려주면 출력하기 편함
        return actual_damage

    def attack_target(self, target: "Character") -> int:
        """
        상대(target)를 공격합니다.
        여기서는 간단하게 내 공격력(self.attack)을 그대로 데미지로 사용합니다.
        (치명타, 랜덤 데미지 같은 건 과제 범위 밖이므로 단순하게!)
        """
        # 상대가 받는 피해량 계산 + hp 감소는 target.take_damage에서 처리
        actual_damage = target.take_damage(self.attack)

        # 이번 공격에서 실제로 들어간 피해량 반환
        return actual_damage


# --------------------------------------------
# [Q2] Player 클래스: Monster 클래스 만들기
# --------------------------------------------
class Player(Character):
    def __init__(self, name: str):
        # 과제 요구 초기값:
        # 레벨 1, 체력 100, 공격력 25, 방어력 5
        super().__init__(name=name, level=1, hp=100, attack=25, defense=5)

        # Player만 추가로 가지는 속성: 경험치
        self.experience = 0

    def gain_experience(self, amount: int):
        """
        인수로 받은 정수(amount)만큼 경험치를 획득합니다.
        """
        # 경험치는 그냥 더해주면 됩니다.
        self.experience += amount

        # 초보자 스타일: 현재 경험치 상태 출력해보기
        print(f"[경험치 획득] {self.name}가 경험치 {amount}을(를) 얻었습니다! (현재 경험치: {self.experience})")

    def level_up(self):
        """
        현재 경험치가 50 이상이면:
        - 레벨 +1
        - 공격력 +10
        - 방어력 +5

        여기서 중요한 점:
        경험치가 100, 150... 처럼 많이 쌓이면 레벨업을 여러 번 할 수도 있습니다.
        과제 설명이 "50 이상이면 레벨을 1 올린다"라고 되어 있어서,
        보통은 '50마다 레벨업 가능'하게 만드는 것이 자연스럽습니다.
        그래서 while로 여러 번 레벨업이 가능하게 만들겠습니다.
        """
        # 경험치가 50 이상이면 레벨업 가능
        while self.experience >= 50:
            # 경험치 50 사용(차감)
            self.experience -= 50

            # 레벨 업!
            self.level += 1
            self.attack += 10
            self.defense += 5

            # 레벨업 안내 출력
            print("====================================")
            print(f"[레벨업!] {self.name}의 레벨이 올랐습니다!")
            print(f"  - 현재 레벨: {self.level}")
            print(f"  - 현재 공격력: {self.attack}")
            print(f"  - 현재 방어력: {self.defense}")
            print(f"  - 남은 경험치: {self.experience}")
            print("====================================")


# --------------------------------------------
# [Q2] Monster 클래스: Character 상속
# --------------------------------------------
class Monster(Character):
    def __init__(self, name: str, level: int):
        # 몬스터 스탯은 "레벨에 비례"해서 랜덤으로 정합니다.
        # 체력: (10~30 랜덤 정수) * 레벨
        # 공격력: (5~20 랜덤 정수) * 레벨
        # 방어력: (1~5 랜덤 정수) * 레벨

        # 랜덤 범위에서 뽑기
        base_hp = random.randint(10, 30)
        base_attack = random.randint(5, 20)
        base_defense = random.randint(1, 5)

        # 레벨에 비례하여 최종 스탯 계산
        hp = base_hp * level
        attack = base_attack * level
        defense = base_defense * level

        # 부모 클래스(Character) 생성자 호출
        super().__init__(name=name, level=level, hp=hp, attack=attack, defense=defense)


# --------------------------------------------
# [Q3] battle 함수 만들기
# --------------------------------------------
def battle(player: Player, monster: Monster):
    """
    Player 인스턴스와 Monster 인스턴스를 인수로 받아
    둘 중 하나의 체력이 0 이하가 될 때까지 공격을 주고 받는 함수

    - Player가 살아남으면:
      경험치 += (몬스터 레벨 * 20)
      player.level_up() 호출
      '전투 승리!' 출력

    - Player가 살아남지 못하면:
      '전투 패배..' 출력
    """

    print("\n====================================")
    print(f"[전투 시작] {player.name}(Lv.{player.level}) vs {monster.name}(Lv.{monster.level})")
    print(f"  - {player.name} HP: {player.hp}, ATK: {player.attack}, DEF: {player.defense}")
    print(f"  - {monster.name} HP: {monster.hp}, ATK: {monster.attack}, DEF: {monster.defense}")
    print("====================================")

    # 라운드 번호를 만들어서 출력
    round_num = 1

    # 둘 중 하나가 죽을 때까지 반복
    while player.is_alive() and monster.is_alive():
        print(f"\n----- [라운드 {round_num}] -----")

        # 1) 플레이어가 몬스터를 공격
        damage_to_monster = player.attack_target(monster)
        print(f"{player.name}의 공격! {monster.name}에게 {damage_to_monster} 피해!")
        print(f"{monster.name}의 남은 HP: {monster.hp}")

        # 몬스터가 죽었으면 더 이상 몬스터 턴 진행하지 않음
        if not monster.is_alive():
            break

        # 2) 몬스터가 플레이어를 공격
        damage_to_player = monster.attack_target(player)
        print(f"{monster.name}의 공격! {player.name}에게 {damage_to_player} 피해!")
        print(f"{player.name}의 남은 HP: {player.hp}")

        # 다음 라운드로
        round_num += 1

    # 전투 결과 처리
    if player.is_alive():
        # 경험치 보상: 몬스터 레벨 * 20
        reward_exp = monster.level * 20
        player.gain_experience(reward_exp)

        # 레벨업 판정 및 처리
        player.level_up()

        # 승리 메시지 출력
        print("\n전투 승리!")
    else:
        # 패배 메시지 출력
        print("\n전투 패배..")


# --------------------------------------------
# [Q4] main 함수 만들기
# --------------------------------------------
def main():
    # 몬스터의 이름, 레벨이 매핑된 딕셔너리 정의하기 
    monster_dict = {"슬라임": 1, "고블린": 2, "오크": 3}

    # 플레이어 이름 입력 받기
    user_input = input("플레이어 이름을 입력하세요 (예: 알파): ").strip()

    # 입력이 없으면 "알파"
    if user_input == "":
        player_name = "알파"
    else:
        # 플레이어 이름은 알파로 고정(초보자 과제 느낌)
        player_name = "알파"

    # Player 인스턴스 생성
    player = Player(name=player_name)

    print("\n====================================")
    print(f"[게임 시작] 플레이어 생성 완료: {player.name}")
    print(f"  - 레벨: {player.level}")
    print(f"  - HP: {player.hp}")
    print(f"  - 공격력: {player.attack}")
    print(f"  - 방어력: {player.defense}")
    print("====================================\n")

    # ----------------------------------------
    # 몬스터 3마리와 모두 전투해야 함
    # 단, 전투 도중 Player 사망 시 즉시 중단하고 게임 종료
    # ----------------------------------------

    # (예: 베타-슬라임, 감마-고블린, 델타-오크)
    greek_names_for_monsters = ["베타", "감마", "델타"]

    monster_order = ["슬라임", "고블린", "오크"]

    for i, monster_key in enumerate(monster_order):
        # 플레이어가 이미 죽었으면 더 이상 진행하지 않음
        if not player.is_alive():
            print("\n게임오버")
            return  # main 종료

        # 몬스터 레벨 가져오기
        monster_level = monster_dict[monster_key]

        # 몬스터 이름을 그리스 문자로 구성
        # 예: "베타 슬라임"
        monster_name = f"{greek_names_for_monsters[i]} {monster_key}"

        # Monster 인스턴스 생성
        monster = Monster(name=monster_name, level=monster_level)

        # 전투 진행
        battle(player, monster)

        # 전투 후 플레이어 상태 요약 출력
        if player.is_alive():
            print("\n------------------------------------")
            print(f"[전투 후 상태] {player.name}")
            print(f"  - 레벨: {player.level}")
            print(f"  - HP: {player.hp}")
            print(f"  - 공격력: {player.attack}")
            print(f"  - 방어력: {player.defense}")
            print(f"  - 경험치: {player.experience}")
            print("------------------------------------\n")
        else:
            # 살아있지 않으면 즉시 게임오버 처리
            print("\n게임오버")
            return

    # 3마리 모두 이겼으면 클리어 메시지
    print("몬스터 없음!!! 게임 종료") 
    
if __name__ == "__main__":
    main() 
    


[게임 시작] 플레이어 생성 완료: 알파
  - 레벨: 1
  - HP: 100
  - 공격력: 25
  - 방어력: 5


[전투 시작] 알파(Lv.1) vs 베타 슬라임(Lv.1)
  - 알파 HP: 100, ATK: 25, DEF: 5
  - 베타 슬라임 HP: 11, ATK: 11, DEF: 1

----- [라운드 1] -----
알파의 공격! 베타 슬라임에게 24 피해!
베타 슬라임의 남은 HP: 0
[경험치 획득] 알파가 경험치 20을(를) 얻었습니다! (현재 경험치: 20)

전투 승리!

------------------------------------
[전투 후 상태] 알파
  - 레벨: 1
  - HP: 100
  - 공격력: 25
  - 방어력: 5
  - 경험치: 20
------------------------------------


[전투 시작] 알파(Lv.1) vs 감마 고블린(Lv.2)
  - 알파 HP: 100, ATK: 25, DEF: 5
  - 감마 고블린 HP: 24, ATK: 24, DEF: 8

----- [라운드 1] -----
알파의 공격! 감마 고블린에게 17 피해!
감마 고블린의 남은 HP: 7
감마 고블린의 공격! 알파에게 19 피해!
알파의 남은 HP: 81

----- [라운드 2] -----
알파의 공격! 감마 고블린에게 17 피해!
감마 고블린의 남은 HP: 0
[경험치 획득] 알파가 경험치 40을(를) 얻었습니다! (현재 경험치: 60)
[레벨업!] 알파의 레벨이 올랐습니다!
  - 현재 레벨: 2
  - 현재 공격력: 35
  - 현재 방어력: 10
  - 남은 경험치: 10

전투 승리!

------------------------------------
[전투 후 상태] 알파
  - 레벨: 2
  - HP: 81
  - 공격력: 35
  - 방어력: 10
  - 경험치: 10
------------------------------------


[전투 시작] 알파(Lv.2) vs 델타 오크(L